CARGA

In [ ]:
import pandas as pd

# Cargar el dataset
file_path = 'cdc_raw_data.csv'
df = pd.read_csv(file_path)

# Mostrar las primeras filas y la información del dataset
print("Dimensiones del dataset:", df.shape)
print("Primeras filas:\n", df.head())
print("Información del dataset:\n")
df.info()


Detección y Tratamiento de Valores Faltantes

In [ ]:
# Resumen de los valores faltantes
missing_data = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100
missing_summary = pd.DataFrame({'missing_count': missing_data, 'missing_percentage': missing_percentage})

print("Valores faltantes:\n", missing_summary)

# Dado que no hay muchas columnas, se puede optar por eliminar las filas con valores faltantes
df_cleaned = df.dropna()
print("Dimensiones después de limpiar valores nulos:", df_cleaned.shape)


Análisis Descriptivo de Variables

In [ ]:
# Resumen estadístico de las variables numéricas
print("\nResumen estadístico de variables numéricas:\n", df_cleaned.describe())

# Distribución de variables categóricas
print("\nDistribución de 'cause_name':\n", df_cleaned['cause_name'].value_counts())

Detección de Outliers

In [ ]:
# Función para detectar outliers usando IQR
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

# Detectar outliers en 'deaths' y 'aadr'
outliers_deaths = detect_outliers(df_cleaned, 'deaths')
outliers_aadr = detect_outliers(df_cleaned, 'aadr')

print("Outliers detectados en 'deaths':\n", outliers_deaths)
print("Outliers detectados en 'aadr':\n", outliers_aadr)

Visualización de Datos

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Histograma de la variable 'deaths'
plt.figure(figsize=(10, 6))
sns.histplot(df_cleaned['deaths'], bins=30, kde=True)
plt.title('Distribución de Muertes')
plt.show()

# Gráfico de barras para 'state'
plt.figure(figsize=(10, 6))
sns.countplot(x='state', data=df_cleaned, order=df_cleaned['state'].value_counts().index)
plt.xticks(rotation=90)
plt.title('Distribución por Estado')
plt.show()

# Matriz de correlación
plt.figure(figsize=(8, 6))
sns.heatmap(df_cleaned[['deaths', 'aadr']].corr(), annot=True, cmap='coolwarm')
plt.title('Matriz de Correlación entre Muertes y AADR')
plt.show()

Distribución de muertes por año

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='year', data=df_cleaned, order=df_cleaned['year'].value_counts().sort_index().index)
plt.title('Número de Muertes por Año')
plt.xticks(rotation=45)
plt.show()

Principales 10 causas de muerte en los últimos 5 años

In [ ]:
recent_years = df_cleaned[df_cleaned['year'] >= 2015]
top_causes = recent_years['cause_name'].value_counts().nlargest(10).index

plt.figure(figsize=(12, 6))
sns.countplot(y='cause_name', data=recent_years[recent_years['cause_name'].isin(top_causes)], 
              order=recent_years['cause_name'].value_counts().nlargest(10).index)
plt.title('Principales 10 Causas de Muerte (2015-2020)')
plt.show()

Tendencias de muertes para las principales causas (agrupación por año)

In [ ]:
# Tendencias de muertes para las principales causas
plt.figure(figsize=(12, 6))
for cause in top_causes:
    cause_trend = recent_years[recent_years['cause_name'] == cause].groupby('year')['deaths'].sum()
    sns.lineplot(x=cause_trend.index, y=cause_trend.values, label=cause)

plt.title('Tendencias de Muertes para las Principales Causas (2015-2020)')
plt.ylabel('Número de Muertes')
plt.xlabel('Año')
plt.legend(title='Causa')
plt.show()

Análisis de correlación detallada entre muertes y AADR

In [ ]:
plt.figure(figsize=(10, 8))
sns.pairplot(df_cleaned[['deaths', 'aadr']])
plt.suptitle('Correlación entre Muertes y Tasa Ajustada de Edad', y=1.02)
plt.show()

Muertes agrupadas por estado y causa

In [ ]:
state_cause_group = df_cleaned.groupby(['state', 'cause_name'])['deaths'].sum().unstack().fillna(0)
print("Muertes agrupadas por Estado y Causa:\n", state_cause_group.head())

Boxplot de la tasa ajustada de mortalidad por edad (AADR) por año


In [ ]:
# Boxplot de AADR por año
plt.figure(figsize=(10, 6))
sns.boxplot(x='year', y='aadr', data=df_cleaned)
plt.title('Boxplot de Tasa Ajustada de Mortalidad por Edad (AADR) por Año')
plt.xticks(rotation=45)
plt.show()

Exportar

In [ ]:
output_file_path = "cleaned_cdc_data.csv"  # Puedes cambiar la ruta si es necesario
df_cleaned.to_csv(output_file_path, index=False)
print(f"Datos limpios exportados a {output_file_path}")